In [ ]:
import json
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import scipy.signal as signal

def cargar_y_filtrar_json(input_file, fs, lowcut=0.5, highcut=40.0, notch_freq=50.0, Q=30.0, order=4):
    with open(input_file, "r") as f:
        data_json = json.load(f)

    signal_raw = np.array([item["value"] for item in data_json], dtype=np.float32)
    
    if len(signal_raw) == 0:
        raise ValueError("SIGNAL IS EMPTY")

    b, a = signal.butter(order, [lowcut/(0.5*fs), highcut/(0.5*fs)], btype='band')
    signal_filtered = signal.filtfilt(b, a, signal_raw)

    b_notch, a_notch = signal.iirnotch(notch_freq, Q, fs)
    signal_filtered = signal.filtfilt(b_notch, a_notch, signal_filtered)

    scaler = MinMaxScaler(feature_range=(0, 1))
    signal_normalizada = scaler.fit_transform(signal_filtered.reshape(-1, 1)).flatten()
    
    return signal_normalizada

def segmentar_y_guardar(signal_filtered, segment_length, output_file):
    num_segments = int(np.ceil(len(signal_filtered) / segment_length))
    
    padded_length = num_segments * segment_length
    signal_padded = np.pad(signal_filtered, (0, padded_length - len(signal_filtered)), 
                           mode='constant', constant_values=0)
    
    segments = signal_padded.reshape(num_segments, segment_length)
    
    np.savetxt(output_file, segments, delimiter=';', fmt='%.6f')
    print(f"✅ Guardado {num_segments} segmentos en {output_file} (incluye padding si hizo falta)")

if __name__ == "__main__":
    input_file = "ecg_data_to_load.txt"  
    output_file = "ecg_segmentado_187.csv"
    fs = 125         
    segment_length = 187

    signal_filtered = cargar_y_filtrar_json(input_file, fs)
    segmentar_y_guardar(signal_filtered, segment_length, output_file)


Archivo de segmentos guardado: ecg_segmentado_187.csv
